Слой батч-нормализации существует для входа любой размерности.

В данном task4 рассмотрим его для входа из многоканальных двумерных тензоров, например, изображений.

Если вытянуть каждый канал картинки в вектор, то вход будет трехмерным:

    количество картинок в батче
    число каналов в каждой картинке
    число пикселей в картинке



![alt text](batchn.png)

Процесс нормализации:

    Вход разбивается на срезы, параллельные синей части. То есть, каждый срез - это все пиксели всех изображений по одному из каналов.
    Для каждого среза считаются мат. ожидание и дисперсия.
    Каждый срез нормализуется независимо.

 

<b><u>Задача</u></b>: реализовать батч-норм слой для четырехмерного входа (например, батч из многоканальных двумерных картинок) без использования стандартной реализации со следующими упрощениями:

    Параметр Бета = 0.
    Параметр Гамма = 1.
    Функция должна корректно работать только на этапе обучения.


In [67]:
import torch
import torch.nn as nn

eps = 1e-3

input_channels = 3
batch_size = 3
height = 10
width = 10

batch_norm_2d = nn.BatchNorm2d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, height, width, dtype=torch.float)


def custom_batch_norm2d(input_tensor, eps):
    arraysD = [input_tensor[:, i, ...].var(unbiased=False) for i in range(input_channels)]
    M = input_tensor.mean(dim=(0, 2, 3)).view(1, -1, 1, 1)
    D = torch.tensor(arraysD).view(1, -1, 1, 1)
    normed_tensor = (input_tensor - M) / torch.sqrt(D + eps)
    return normed_tensor



In [68]:
# Проверка происходит автоматически вызовом следующего кода

norm_output = batch_norm_2d(input_tensor)
custom_output = custom_batch_norm2d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)

True
